In [12]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
from pathlib import Path
import pandas as pd
from os.path import exists

##this block creates functions that can be used to extract the data from the steam API
##maybe we can add this block in the source file as a functions file

def get_request(url, parameters=None):
    #function to create an API request based on URL and  parameters
    response = requests.get(url=url, params=parameters)
    if response:
        time.sleep(1)
        return response.json()
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)

def getAppList():
    #using the function to get all data id's which include appid and app name 
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    parameters = {"request": all}
    return get_request(url, parameters=parameters)
    print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")
    
def getAppDetails(id):
    #function to collect the raw data from the api per app_id
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {id}}
    raw_data = get_request(url, parameters=parameters)
    response = raw_data[str(id)]
    if(response['success']):
        data_json = response["data"]
        data_json['collection_details'] = {'created_by': 'our_scraper',
                                            'created_at': datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
        f = open('..\\..\\data\\raw_data.json','a',encoding='utf-8')
        f.write(json.dumps(data_json)+'\n')
        f.close()
        return data_json
    return 
 

In [2]:
#collect all app id's from steam 
data_ids = getAppList()

#Code to collect all app id's in one list and write to a json file called app_ids 
all_app_ids = []
print("Creating a list of all IDs currently available on Steam")

for item in data_ids['applist']['apps']:
    all_app_ids.append(item['appid'])

f = open('..\\..\\data\\app_ids.json','w',encoding='utf-8')
f.write(json.dumps(all_app_ids)+'\n')
f.close()

print("List created sucessfully, the total number of IDs available on Steam is:", len(all_app_ids),".")

Your request is being processed.
Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138682 .


In [13]:
#This code can be run to collect the appids, in case there was data collected earlier in the raw_data.json file than it will add to existing files. 

#put the amount of appids you want to collect in the 'limit'parameter
counter = 0
limit = 500

## collect app_ids from the  app_ids file created before 
r = open('..\\..\\data\\app_ids.json','r',encoding='utf-8')
app_ids = json.load(r)

## creates an empty raw_data.json that can be appended on
f = open('..\\..\\data\\raw_data.json','a',encoding='utf-8') 

# check if the json from earlier was created and set limit 
raw_exists = exists("..\\..\\data\\raw_data.json")

#create list of already collected id's 
if (raw_exists):
    #create list of collected id's 
    col_ids = []
    with open('..\\..\\data\\raw_data.json') as f: 
        for jsonObj in f: 
            rawDict = json.loads(jsonObj)
            col_ids.append(rawDict["steam_appid"])

## getting the new data
#  keep in mind that while running the code it can reach a limit, letting it run it automaticly continues after a minute or so 
for i in app_ids: 
    if i not in col_ids:
        if(counter > limit):
            break
        id = str(i)
        counter = counter + 1
        app_data = getAppDetails(i)
        print("Processing data pertaining to Steam ID:"+id)
    else:
        print(f"Skipping id {str(i)}")

Processing data pertaining to Steam ID:1828741
Processing data pertaining to Steam ID:216938
Processing data pertaining to Steam ID:660010
Processing data pertaining to Steam ID:660130
Processing data pertaining to Steam ID:1118314
Processing data pertaining to Steam ID:1275822
Processing data pertaining to Steam ID:1343832
Processing data pertaining to Steam ID:662172
Processing data pertaining to Steam ID:1360782
Processing data pertaining to Steam ID:1820332
Skipping id 1877200
Skipping id 1877220
Skipping id 1877250
Skipping id 1877300
Skipping id 1877310
Skipping id 1877330
Skipping id 1877350
Skipping id 1877360
Skipping id 1877370
Processing data pertaining to Steam ID:1877380
Skipping id 1877410
Skipping id 1877440
Skipping id 1877450
Skipping id 1877460
Skipping id 1877470
Skipping id 1877480
Skipping id 1877510
Skipping id 1877520
Skipping id 1877530
Skipping id 1877540
Skipping id 1877550
Skipping id 1877580
Skipping id 1877620
Skipping id 1877650
Skipping id 1877710
Skippin

In [ ]:
#This code takes the raw_data.json output and parses it based on the keys given in the code. 
Allkeys = ['ids', 'raw', 'categories','package_groups','genres','screenshots','movies', 'achievements.highlighted']
dataframes = {}

##creates dataframes for the seperate parsing keys 
for k in Allkeys:
    file = f"..\\..\\data\\dataframe_{k}.xlsx"
    try:
        dataframes[k] = pd.read_excel(file)
    except:
        print(f"File doesn't exist {file}")
        dataframes[k] = pd.DataFrame()
        pass

# check if the json from earlier was created 
raw_exists = exists("..\\..\\data\\raw_data.json")

#collect raw_data.json as object
if (raw_exists):
    app_data = []
    with open('..\\..\\data\\raw_data.json') as f: 
        for jsonObj in f: 
            data = json.loads(jsonObj)
            app_data.append(data)

## parsing the raw_data.json
if(app_data):
    result = pd.json_normalize(app_data)
    for key in Allkeys:
        if( key not in dataframes):
            dataframes[key] = pd.DataFrame()
        try:
            print(f"Parsing for key {key}")

            data = pd.json_normalize(app_data,record_path = [key], meta = ["steam_appid"], errors="ignore")
            dataframes[key] = pd.concat([dataframes[key],data])            
            print(key)
        except:
            pass
else: 
    print("no app_data")                  
    
#writing the new data to the .xlsx files from the new dataframes
for key in Allkeys:
    print(f"Writing to dataframe {key}")
    file = f"..\\..\\data\\dataframe_{key}.xlsx"
    print(dataframes[key])
    dataframes[key].to_excel(file)

In [32]:
#debug block

len(app_data[1]['package_groups'])
len(app_data[1]['categories'])

for i in app_data:
    jsonobj = i['categories']

    for cat in jsonobj:
        f=open('','w')
        #f.write(json[''])

#data = pd.json_normalize(app_data,record_path = ["package_groups"], meta = ["steam_appid"], errors="ignore")



#app_data

8